In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../') # import modules from ../

In [82]:
import weasyprint as wp
import jinja2
import calendar
import logging

from datahandling import *
from graphing import *

import IPython.core.display as ipd

mpl.style.use('seaborn-bright')#'fivethirtyeight')
mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['figure.figsize'] = (8,12) #(3,2)
mpl.rcParams['axes.titlesize'] = 'large'
mpl.rcParams['axes.labelsize'] = 'small'
mpl.rcParams['xtick.labelsize'] = 'small'
mpl.rcParams['ytick.labelsize'] = 'small'
mpl.rcParams['legend.fontsize'] = 'small'
mpl.rcParams['legend.frameon'] = False
mpl.rcParams['legend.shadow'] = True

In [83]:
logging.getLogger().setLevel(logging.INFO)

In [4]:
location_name="Open Lab"
input_datafile="../testdata/fetch.bax"
output_filename='weasyprint_output.pdf'
map_filename='/Users/sam/Dropbox/PhD/Auditing/maps/merz-ceam.svg'
description_text="Report automatically generated using data from environmental sensor deployments in Merz court."

In [5]:
pd.set_option('chained_assignment', None) # Hush up, SettingWithCopyWarning
df = readfile(input_datafile)

In [6]:
t_start, t_end = (df.index.min(), df.index.max())
dfs = split_by_id(df)
names = unique_sensors(df)
lengths = [len(dfs[name]) for name in names]

dfs = fix_humidity(dfs)
dfs = diff_pir(dfs)
dfs = fix_temp(dfs)

[l for l in zip(names, lengths)]

[('80418A51', 18923),
 ('80524F69', 17769),
 ('80B207A1', 19413),
 ('80F82619', 7525)]

In [7]:
dfs[list(dfs.keys())[0]]['Temp'][:4]

DateTime
2016-03-28 00:13:29    12.4
2016-03-28 00:17:50    12.4
2016-03-28 00:18:23    12.4
2016-03-28 00:19:28    12.4
Name: Temp, dtype: float64

In [73]:
print(t_start, 'to', t_end)

# Generate date range of weeks inclusive of start and end
weeks = [day for day in pd.date_range((t_start - pd.Timedelta('7 days')), t_end + pd.Timedelta('7 days'), freq='W-MON', normalize=True, closed=None)]
weeks = [(start,end- pd.Timedelta('1 day')) for start,end in zip(weeks,weeks[1:])]
# Skip weeks with no data
weeks = [ k for k in [w if sum( [len( df[w[0]:w[1]] )] ) > 0 else None for w in weeks] if k is not None ]
weeks

2016-03-27 23:59:53 to 2016-04-03 23:59:39


[(Timestamp('2016-03-28 00:00:00', offset='W-MON'),
  Timestamp('2016-04-03 00:00:00', offset='W-MON'))]

In [84]:
#%%capture
types=[("Temp", "Temperature ˚C"), ("Humidity", "Humidity %RH"), ("Light", "Light (lux)")]
figs=[]

for typestrings in types:
    figs.append( [weekly_graph( dfs, *typestrings, *period ) for period in weeks] )

INFO:graphing.py:Temp     - 2016-03-28 00:00:00 to 2016-04-03 00:00:00
INFO:graphing.py:Humidity - 2016-03-28 00:00:00 to 2016-04-03 00:00:00
INFO:graphing.py:Light    - 2016-03-28 00:00:00 to 2016-04-03 00:00:00


In [85]:
#figs_bkp = figs
#figs2 = [[data[-50:] for data in series] for series in figs]
print(t_start.date())
to_plot = \
[
    [
        {
            'type'    : t,
            'label'   : l,
            'data'    : d[i],
            't_start' : w[0].date(),
            't_end'   : w[1].date()
        } for i,w in enumerate(weeks)
    ] for t,l,d in zip(*zip(*types), figs)
]

# Comes out as: to_plot[week][series][data]
# e.g. to_plot[0][0]['label'] == 'Temperature ˚C'

2016-03-27


In [89]:
with open('../ipynb_mplParams.txt', 'w+') as f:
    f.write("\n".join([ str(k) +' '+ str(v) for k,v in mpl.rcParams.items() ]))

In [86]:
import base64
print(base64.b64decode(to_plot[0][0]['data']).decode('utf8').split('\n')[4])
#ipd.HTML("<img src='data:image/svg+xml;charset=utf-8;base64,%s' />" % figs[0])
ipd.HTML(base64.b64decode(to_plot[0][0]['data']).decode('utf8'))

<svg height="648pt" version="1.1" viewBox="0 0 497 648" width="497pt" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">


In [ ]:
map_b64 = None
if map_filename is not None:
    with open(map_filename, 'rb') as t:
        map_b64 = base64.b64encode(t.read()).decode('utf8').replace('\n','')
    map_b64[:100]

In [ ]:
report_template = '''
{% for series in to_plot %}
<div class="cover">
    <h1> {{ series[0].label.split(' ')[0] }} Report: {{ location }} </h1>
    <h2> Weeks beginning {{ period[0] }} to {{ period[1] }} </h2>
{% if map_b64 is not none %}   
    <img src="data:image/svg+xml;charset=utf-8;base64,{{ map_b64 }}" />
{% endif %}
    <p class="description">{{ description }}</p>
</div>
{% for week in series %}
<h2>{{week.label}} @ {{location}}</h1>
<h3>Time period: 
    <em>{{ week.t_start.strftime('%Y-%m-%d') }} to {{ week.t_end.strftime('%Y-%m-%d') }}</em>
</h3>

<figure>
    <img src="data:image/svg+xml;charset=utf-8;base64,{{week.data}}" />
    <!--<figcaption></figcaption>-->
</figure>
{% endfor %}
{% endfor %}
'''

print_css = wp.CSS(string='''
@page { size: A4 portrait; margin:1cm 2cm; }
@media print {
    .cover {
        height: 100%;
        padding: 0;
        margin: 0;
        page-break-after: always;
        page-break-before: always;
    }
    .cover:first-child {
        page-break-before: avoid;
    }
    .cover img {
        width:80%;
        margin:0;
        padding:0;
    }
}

html { font-family: sans; }
html, body {
    width: 100%;
    margin: 0;
    padding: 0;
    position: relative;
}
figure {
    padding:0;
    margin:0;
    width:100%;
}
img { width: 100%; }
''')

debug_css = wp.CSS(string='''
* { box-sizing: border-box; outline: solid 2px #900; }
* > * { outline-color: #090; }
* > * > * { outline-color: #009; }
''')

variables = {
    'location': location_name,
    'map_b64' : map_b64 or None,
    'description' : description_text,
    'period'  : ( weeks[0][0].date().strftime('%d %b'), weeks[-1:][0][0].date().strftime('%d %b') ),
    'to_plot' : to_plot,
}
output = jinja2.Template(report_template).render(**variables)

In [ ]:
#with open('test.htm', 'w+') as t: t.write(output)
#ipd.display(ipd.HTML(output))

In [ ]:
# write to PDF
htm = wp.HTML(string=output, base_url='.')
pdf = htm.write_pdf(target=output_filename, zoom=2, stylesheets=[print_css])#, debug_css])

In [ ]:
#!open weasyprint_output.pdf